# Search Engine full stack 

## Building an Indexer for the crawled webpages

In [1]:
import os, elasticsearch

In [2]:
scrapped = {}

article_path = "newyorktimes/newyorktimes/articles"

docs = [] 
doc_names = []
doc_dic ={}
for root, dirs, files in os.walk(article_path):
    for file in files:
        with open(os.path.join(root,file), 'r') as f:
            doc_dic[file]= f.read()
            
docs = [*doc_dic.values()]
doc_names = [*doc_dic.keys()] 

In [3]:
es = elasticsearch.Elasticsearch()

for i in range(len(docs)):
    title = doc_names[i].split('-')[1].split('.txt')[0]
    authors = list(filter(lambda x: x, docs[i].split('Author(s): ')[1].split('\n')[0].split(', ')))
    url = docs[i].split('url: ')[1].split('\n')[0]
    text =''.join(docs[i].split('url: ')[1].split('\n')[2:])

    es.index(index='scrapped', doc_type='article', id=i, body={
        'title': title,
        'authors': authors,
        'url': url,
        'text': text
    })

In [4]:
def search_exact_word(word):
    res = es.search(index='scrapped', body={"query": {"match": {"text": word}}})
    return list(map(lambda x: x['_source']['title'], res['hits']['hits']))